# Analyse d'une collecte de mémoire vive

In [ ]:
from colorama import init, Fore, Back, Style
import os
import json
import pandas as pd
import re
from IPython.display import display, HTML

from pandas.io.json import json_normalize
import ipaddress as ip
from msticpy.transform.iocextract import IoCExtract
import msticpy as mp
mp.init_notebook(globals(), verbosity=0)
ti = mp.TILookup()
ioc_extract = IoCExtract()

#Expand the width of the cells
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def ip_type(string):
    if ip.ip_address(string).is_private:
        return 'Private'
    elif ip.ip_address(string).is_multicast:
        return 'Multicast'
    elif ip.ip_address(string).is_reserved:
        return 'Reserved'
    elif ip.ip_address(string).is_loopback:
        return 'Loopback'
    elif ip.ip_address(string).is_global:
        return 'Public'
    elif ip.ip_address(string).is_link_local:
        return 'Link local'


mem_path = "/home/secubian/Desktop/Cases/0C-002-memdump.mem"
vol  = f"vol --offline -f {mem_path}"


## Récupération d'informations relatives au prélévement mémoire

Cette étape est importante pour la suite de l'investigation.
Elle va permettre de vérifier que le profile correspondant au système d'exploitation du prélévement est bien pris en charge par Volatility3. 

In [ ]:
os.system(f"{ vol } windows.info.Info")

# Extraction des informations pour analyse

Cette étape va permettre d'exécuter un certain nombre de modules pour extraire l'ensemble des informations nécessaires à l'investigation de l'analyste.

In [ ]:
volplugins_lst = ["windows.privileges.Privs", "windows.sessions.Sessions","windows.pslist.PsList","windows.psscan.PsScan","windows.cmdline.CmdLine","windows.netscan.NetScan","windows.callbacks.Callbacks","windows.driverscan.DriverScan","windows.dlllist.DllList","windows.svcscan.SvcScan","windows.handles.Handles","windows.malfind.Malfind","windows.ssdt.SSDT"]

for plugin in volplugins_lst:
    # Check if plugin is already executed
    if not (os.path.isfile(f"{ mem_path }_{ plugin }.json")):
        # Execute plugin and save result in json file
        os.system( f"{ vol } -r json  { plugin } > { mem_path }_{ plugin }.json")

    globals()[f"df_{ plugin.replace('.','_') }"] = pd.read_json(f"{ mem_path }_{ plugin }.json")



## Enrichissement des données collectées

In [ ]:
dfsession = df_windows_sessions_Sessions.rename(columns={'Process ID': 'PID'})
#dfsession[['PID', 'User Name']]

# Enrichissement des processus - PsList
df_windows_pslist_PsList_enriched = df_windows_pslist_PsList.merge(dfsession, on='PID', how='left' )
dfppid = df_windows_pslist_PsList_enriched[['PID', 'ImageFileName','User Name']].drop_duplicates()
dfppid.columns = ['PPID', 'ParentProcessName','ParentUserName']
df_windows_pslist_PsList_enriched = df_windows_pslist_PsList_enriched.merge(dfppid, on='PPID', how='left')
#df_windows_pslist_PsList_enriched.sort_values(by=['SessionId','PPID','ParentProcessName'])[['SessionId','PID','ImageFileName','User Name','PPID','ParentProcessName','ParentUserName']].head()


# Enrichissement des processus - PsScan
df_windows_psscan_PsScan_enriched = df_windows_psscan_PsScan.merge(dfsession, on='PID', how='left' )
dfppid = df_windows_psscan_PsScan_enriched[['PID', 'ImageFileName','User Name']].drop_duplicates()
dfppid.columns = ['PPID', 'ParentProcessName','ParentUserName']
df_windows_psscan_PsScan_enriched = df_windows_psscan_PsScan_enriched.merge(dfppid, on='PPID', how='left')
#df_windows_psscan_PsScan_enriched.sort_values(by=['SessionId','PPID','ParentProcessName'])[['SessionId','PID','ImageFileName','User Name','PPID','ParentProcessName','ParentUserName']].head()


# Enrichissement des résultats du mdule CmdLine
df_windows_cmdline_CmdLine_enriched = df_windows_cmdline_CmdLine.merge(dfsession, on='PID', how='left' )
df_windows_cmdline_CmdLine_enriched = df_windows_cmdline_CmdLine_enriched.rename(columns={'Process_x': 'Process'})
dfppid = df_windows_psscan_PsScan_enriched[['PID', 'PPID', 'ParentProcessName','ParentUserName']].drop_duplicates()
df_windows_cmdline_CmdLine_enriched = df_windows_cmdline_CmdLine_enriched.merge(dfppid, on='PID', how='left' )

# Enrichissement des Dlls
df_windows_dlllist_DllList_enriched = df_windows_dlllist_DllList.merge(dfsession, on='PID', how='left' )
dfppid = df_windows_psscan_PsScan_enriched[['PID','PPID', 'ParentProcessName','ParentUserName']].drop_duplicates()
df_windows_dlllist_DllList_enriched = df_windows_dlllist_DllList_enriched.merge(dfppid, on='PID', how='left')
df_windows_dlllist_DllList_enriched = df_windows_dlllist_DllList_enriched[['Path','PID','Process_x','User Name','PPID', 'ParentProcessName','ParentUserName']][df_windows_dlllist_DllList_enriched["Path"].str.contains('programdata|users|utilisateurs|temp|tmp', flags=re.IGNORECASE) == True].drop_duplicates()

# Enrichissement des Handles
df_windows_handles_Handles_enriched = df_windows_handles_Handles.merge(dfsession, on='PID', how='left')
dfppid = df_windows_psscan_PsScan_enriched[['PID','PPID', 'ParentProcessName','ParentUserName']].drop_duplicates()
df_windows_handles_Handles_enriched = df_windows_handles_Handles_enriched.merge(dfppid, on='PID', how='left')
df_windows_handles_Handles_enriched = df_windows_handles_Handles_enriched[['Create Time', 'PID', 'Process_x', 'User Name', 'Name', 'Type',  'GrantedAccess','HandleValue', 'Offset',    'PPID', 'ParentProcessName', 'ParentUserName']]

# Enrichissement des résultats du module : NetScan
df_windows_netscan_NetScan_enriched = df_windows_netscan_NetScan.merge(dfsession, on='PID', how='left')
dfppid = df_windows_psscan_PsScan_enriched[['PID','PPID', 'ParentProcessName','ParentUserName']].drop_duplicates()
df_windows_netscan_NetScan_enriched = df_windows_netscan_NetScan_enriched.merge(dfppid, on='PID', how='left')

# Enrichissement des résultats du module : malfind
df_windows_malfind_Malfind_enriched = df_windows_malfind_Malfind.merge(dfsession[['PID','User Name']], on='PID', how='left')
dfppid = df_windows_psscan_PsScan_enriched[['PID','PPID', 'ParentProcessName','ParentUserName']].drop_duplicates()
df_windows_malfind_Malfind_enriched = df_windows_malfind_Malfind_enriched.merge(dfppid, on='PID', how='left')

# Enrichissement des résultats du module : svcscan
df_windows_svcscan_SvcScan = df_windows_svcscan_SvcScan.dropna()
df_windows_svcscan_SvcScan['PID'] = df_windows_svcscan_SvcScan['PID'].astype('int')
df_windows_svcscan_SvcScan_enriched = df_windows_svcscan_SvcScan.merge(dfsession[['PID','User Name']], on='PID', how='left')
dfppid = df_windows_psscan_PsScan_enriched[['PID','PPID', 'ParentProcessName','ParentUserName']].drop_duplicates()
df_windows_svcscan_SvcScan_enriched = df_windows_svcscan_SvcScan_enriched.merge(dfppid, on='PID', how='left')

# Enrichissemnt des résultats du module : privileges
df_windows_privileges_Privs_enriched = df_windows_privileges_Privs.merge(dfsession[['PID','User Name']], on='PID', how='left')
df_windows_privileges_Privs_enriched = df_windows_privileges_Privs_enriched.merge(dfppid, on='PID', how='left')


# Analyse des processus

Cette phase permet d'analyser les processus en cours d'exécution lors du prélévement.
La première étape consiste donc à analyser la double liste chainée répertoriant les processus au sein du système d'exploitation Microsoft Windows.

## Identification de processus cachés

Analyse les différences entre les résultats des plugins PsTree et PsScan.
Malheureusement, il est tout à fait possible de masquer une exécution de processus malveillant, en ne l'intégrant pas dans cette liste. Il va donc faloir utiliser le module ```PsScan``` qui a pour objectif de scanner l'ensemble de la mémoire à la recherche de structure EPROCESS, puis comparer les deux résultats.

In [ ]:
for proc in df_windows_psscan_PsScan_enriched['PID']:
    p_count = df_windows_pslist_PsList[df_windows_pslist_PsList.PID == proc].shape[0]
    if p_count == 0 :
        procPID = proc
        procPPID = df_windows_psscan_PsScan_enriched[df_windows_psscan_PsScan_enriched.PID == proc]['PPID'].to_string(index = False)
        procFileName = df_windows_psscan_PsScan_enriched[df_windows_psscan_PsScan_enriched.PID == proc]['ImageFileName'].to_string(index = False)
        procUserName = df_windows_psscan_PsScan_enriched[df_windows_psscan_PsScan_enriched.PID == proc]['User Name'].to_string(index = False)
        procParentProcessName = df_windows_psscan_PsScan_enriched[df_windows_psscan_PsScan_enriched.PID == proc]['ParentProcessName'].to_string(index = False)
        procParentUserName = df_windows_psscan_PsScan_enriched[df_windows_psscan_PsScan_enriched.PID == proc]['ParentUserName'].to_string(index = False)
        print(Fore.RED + f"[!] Suspicious process hidden in psList for PID: { procPID }, Filename: { procFileName}, executed by Username: {procUserName}, with PPID: {procPPID}, ParentProcessName: {procParentProcessName} executed by: {procParentUserName}")

## Recherche des processus systèmes ayant plus occurrences

- Définition des processus devant être unique sur un système Microsoft Windows 

In [ ]:
singleprocs = ["system","wininit.exe","lsass.exe","lsm.exe","services.exe","lsaiso.exe"]

- Lancement de la recherche

In [ ]:
for procs in singleprocs:
    p_count = df_windows_pslist_PsList[df_windows_pslist_PsList.ImageFileName.str.lower() == procs].shape[0]
    if p_count == 1:
        print(Fore.GREEN + "[✓]No multiple instance(s) of " + procs + " found")
    elif p_count > 1:
        print(Fore.RED + "[!]Multiple instance(s) of " + procs + " found!")
        print(df_windows_pslist_PsList[df_windows_pslist_PsList.ImageFileName.str.lower() == procs][['PID','ImageFileName','UserName','PPID','ParentProcessName','ParentUserName']].to_string(index = False))


## Analyse des processus svchost et rundll32

Tous les processus svchost et rundll32 possèdent des arguments dans un fonctionnement normal. \
Voici donc la liste des processus n'ayant pas d'argument et de ce fait considérés comme suspects.

In [ ]:
noargs = df_windows_cmdline_CmdLine_enriched[df_windows_cmdline_CmdLine_enriched['Args'].str.contains('svchost.exe$|rundll32.exe$', flags=re.IGNORECASE)][['PID','Process','Args','User Name']]
if (noargs.empty):
    print(Fore.GREEN + "[✓] No svchost.exe or rundll32.exe process identified without commandline arguments")
else:
    print(Fore.RED + "[!] Some svchost.exe or rundll32.exe without command line was identified!")
    print(noargs.to_string(index = False))
    

## Analyse des processus connus exécutés à partir de chemin non standards

Certains outils intégrés par défaut au sein du système d'exploitation Microsoft peuvent être très utils durant une attaque. Malheureusement ces outils sont très souvent surveillés. C'est pour cela, que certains modes opératoires utilisent ces outils à partir de répertoires temporaires, ou dans les répertoires correspondant aux profiles des utilisateurs compromis. La recherche ci-dessous va permettre de comparer les chemins connus de ceux actuellement présents dans la collecte de la mémoire vive pour déceler toute incohérence.

In [ ]:
knownProcesses = {
                    'conhost.exe':['c:\windows\system32\conhost.exe'],
                    'dllhost.exe':['c:\windows\system32\dllhost.exe','c:\windows\syswow64\dllhost.exe'],
                    'csrss.exe':['c:\windows\system32\csrss.exe', '%systemroot%\system32\csrss.exe'],
                    'explorer.exe':['c:\windows\explorer.exe','c:\windows\syswow64\explorer.exe'],
                    'lsaiso.exe':['c:\windows\system32\lsaiso.exe'],
                    'lsass.exe':['c:\windows\system32\lsass.exe'],
                    'runtimebroker.exe':['c:\windows\system32\runtimebroker.exe'],
                    'services.exe':['c:\windows\system32\services.exe'],
                    'smss.exe':['c:\windows\system32\smss.exe','\systemroot\system32\smss.exe'],
                    'svchost.exe':['c:\windows\system32\svchost.exe','c:\windows\syswow64\svchost.exe'],
                    'taskhostw.exe':['c:\windows\system32\taskhostw.exe'],
                    'wininit.exe':['c:\windows\system32\wininit.exe', 'wininit.exe'],
                    'winlogon.exe':['c:\windows\system32\winlogon.exe', 'winlogon.exe'],
                    'wmiprvse.exe':['c:\windows\system32\wbem\wmiprvse.exe','c:\windows\syswow64\wbem\wmiprvse.exe']
}

suspProcess = []
for process in df_windows_cmdline_CmdLine_enriched[['Args','Process','PID']].head(20).to_dict('records'):
    processPath = process['Args'].strip().split(' ')[0].lower()
    processName = process['Process'].lower()
    processId   = process['PID']
    try:
        found = 0
        for path in knownProcesses[processName]:
            if path == processPath:
                found = 1
        if found == 0:
            suspProcess.append(process)
    except:
        pass

if suspProcess:
    print(Fore.RED + "[!] Some known Microsoft Windows binaries used from non known default path ")
    for process in suspProcess:
        print(process)
else:
    print(Fore.GREEN + "[✓] No known Microsoft Windows binaries used from non known default path ")


## Analyse des Dlls

Affichage de l'ensemble des librairies systèmes chargées à partir de répertoires non habituels.

In [ ]:
suspdlls = df_windows_dlllist_DllList_enriched[['Path','PID','Process_x','UserName','PPID', 'ParentProcessName','ParentUserName']][df_windows_dlllist_DllList_enriched["Path"].str.contains('.exe$', flags=re.IGNORECASE) == False]
if (suspdlls.empty):
    print(Fore.GREEN + "[✓] No Suspicious DLLs identified")
else:
    print(Fore.RED + "[!] Suspicious DLLs identified")
    print(suspdlls.to_string(index = False))

## Analyse hooking SSDT

In [ ]:
sus_kernelhook = df_windows_ssdt_SSDT[(df_windows_ssdt_SSDT["Module"].str.match(r"(ntoskrnl1|win32k)",case=False) == False)][["Address","Module","Symbol"]]
if (sus_kernelhook.empty):
    print(Fore.GREEN + "[✓]No suspicious SSDT hooking detected")
else:
    print(Fore.RED + "[!]Suspicious SSDT hooking detected!")
    print(sus_kernelhook.to_string(index = False))

## Analyse process identified by malfind

In [ ]:
procinject1=df_windows_malfind_Malfind_enriched[df_windows_malfind_Malfind_enriched["Hexdump"].str.contains("MZ") == True][['PID','Process','UserName','Start VPN','End VPN','Protection','Disasm', 'PPID', 'ParentProcessName', 'ParentUserName']]
if (procinject1.empty):
    print(Fore.GREEN + "[✓]No MZ header detected in malfind preview output")
else:
    print(Fore.RED + "[!]MZ header detected within malfind preview (Process Injection indicator) \n")
    print(procinject1[['Protection', 'PID','Process','UserName', 'PPID', 'ParentProcessName', 'ParentUserName']].drop_duplicates().to_string(index = False))



## [TODO] Analyse des privilèges associés aux processus

## [TODO] Analyse Handle to detect pipe

Les malwares utilisent très souvent des tubes nommés (Pipe Named) afin d'échanger avec le système 

In [ ]:
df_windows_handles_Handles_enriched[(df_windows_handles_Handles_enriched["Type"] == "File") & (df_windows_handles_Handles_enriched["Type"].str.match(r"\\Device\\NamedPipe\\().*",case=False) == False)].head()

## [TODO]  Syscall analysis

# Network Analysis

## Identification de connexions distantes, potentiellement suspectes

Voici une analyse des adresses ip publiques via les bases de Threat Intelligence : 

In [ ]:
# source : 
#os.system('export MSTICPYCONFIG="/home/secubian/Desktop/Documentation/JupyterNotebook/msticpyconfig.yaml"')

#df_windows_netscan_NetScan_public_ip = df_windows_netscan_NetScan_enriched[['ForeignAddr']][(df_windows_netscan_NetScan_enriched["ForeignAddr"] != "*") & (df_windows_netscan_NetScan_enriched["ForeignAddr"] != "0.0.0.0") & (df_windows_netscan_NetScan_enriched["ForeignAddr"] != "::")].drop_duplicates()
#df_windows_netscan_NetScan_public_ip_ioc = ioc_extract.extract(data=df_windows_netscan_NetScan_public_ip, columns=['ForeignAddr'])
#df_windows_netscan_NetScan_public_ip_ioc = df_windows_netscan_NetScan_public_ip_ioc[['IoCType','Observable']].rename(columns={'Observable': 'Ioc'})

suspnetworkconnections = df_windows_netscan_NetScan_enriched[['Create Time', 'State', 'Proto', 'LocalAddr', 'LocalPort', 'ForeignAddr', 'ForeignPort', 'Session Type', 'PID', 'Process', 'UserName',   'PPID','ParentProcessName', 'ParentUserName']][(df_windows_netscan_NetScan_enriched["State"] == "ESTABLISHED") | (df_windows_netscan_NetScan_enriched["State"] == "CLOSED")].sort_values(by='PID', ascending=False)
suspnetworkconnections['ip_type'] = suspnetworkconnections['ForeignAddr'].apply(ip_type)

if (suspnetworkconnections[(suspnetworkconnections["ip_type"] == 'Public')].empty):
    print(Fore.GREEN + "[✓] No Potentially Suspicious Network connections identified")
else:
    df_windows_netscan_NetScan_ti = ti.lookup_iocs(data=suspnetworkconnections[(suspnetworkconnections["ip_type"] == 'Public')], ioc_col="ForeignAddr", providers=["VirusTotal", "OTX"])
    df_suspnetworkconnections = df_windows_netscan_NetScan_ti[df_windows_netscan_NetScan_ti['Result']==True]
    df_suspnetworkconnections = pd.json_normalize(data=df_suspnetworkconnections[['Ioc','Provider','Details']].to_dict(orient='records'))

    if (df_suspnetworkconnections.empty):
        print(Fore.GREEN + "[✓] No Potentially Suspicious Network connections identified")
    else:
        print(Fore.RED + "[!] Potentially Suspicious Network connections identified")
        display(df_suspnetworkconnections)



Voici les connexions réseaux à destination des réseaux privés ou publics.

In [ ]:
suspnetworkconnections = df_windows_netscan_NetScan_enriched[['Create Time', 'State', 'Proto', 'LocalAddr', 'LocalPort', 'ForeignAddr', 'ForeignPort', 'Session Type', 'PID', 'Process', 'UserName',   'PPID','ParentProcessName', 'ParentUserName']][(df_windows_netscan_NetScan_enriched["State"] == "ESTABLISHED") | (df_windows_netscan_NetScan_enriched["State"] == "CLOSED")].sort_values(by='PID', ascending=False)
suspnetworkconnections['ip_type'] = suspnetworkconnections['ForeignAddr'].apply(ip_type)

if (suspnetworkconnections[(suspnetworkconnections["ip_type"] == 'Public') | (suspnetworkconnections["ip_type"] == 'Private')].empty):
    print(Fore.GREEN + "[✓] No Potentially Suspicious Network connections identified")
else:
    print(Fore.RED + "[!] Potentially Suspicious Network connections identified\n")
    print(suspnetworkconnections[(suspnetworkconnections["ip_type"] == 'Public') | (suspnetworkconnections["ip_type"] == 'Private')].to_string(index = False))

## Identification des processus en écoute sur le système

In [ ]:
df_windows_netscan_NetScan_enriched[['Create Time', 'State', 'Proto', 'LocalPort', 'PID', 'Process', 'UserName',   'PPID','ParentProcessName', 'ParentUserName']][(df_windows_netscan_NetScan_enriched["State"] == "LISTENING")].sort_values(by='PID', ascending=False)

# Analyse des services

Cette analyse va permettre d'identifier les services exécutant des binaires à partir d'emplacement non conventionnels.

In [ ]:
suspservice = df_windows_svcscan_SvcScan_enriched.dropna()
suspservice = suspservice[~suspservice["Binary"].str.lower().str.contains(':\\\\windows\\\\system32\\\\|:\\\\program files\\\\')]
if (suspservice.empty):
    print(Fore.GREEN + "[✓] No Potentially Suspicious Services identified")
else:
    print(Fore.RED + "[!] Potentially Suspicious Services identified\n")
    print(suspservice[['Name','Display','Binary','UserName','State','PID','Start']].to_string(index = False))

# Generation de la timeline

La prochaine étape consiste à générer la timeline des actions réalisées sur le système d'exploitation et le système de fichiers afin de déterminer tout comportement malveillant. \
Pour ce faire, nous allons utiliser le module : timeliner.Timeliner \
Une fois la timeline prête, nous l'importerons dans l'instance Timesketch présente sur Secubian.

PS : Il semble y avoir un souci de compatibilité entre la sortie de volatility3 et log2timeline \
https://github.com/volatilityfoundation/volatility3/issues/542

In [ ]:
# Execute plugin and save result in json file
plugin = "timeliner.Timeliner"
#os.system( f"{ vol } { plugin } --create-bodyfile > { mem_path }_{ plugin }.body")
#os.system( f"{ vol } { plugin }   > { mem_path }_{ plugin }.json")

#os.system( f"mactime -z UTC -y -d -b { mem_path }_{ plugin }.body > { mem_path }_{ plugin }.csv")

